In [1]:
import os.path as osp
import GPUtil
import torch.nn as nn
import torch

from torch.nn import TransformerEncoder, TransformerEncoderLayer
import os
from torch_geometric.data import Dataset, download_url
from torch_geometric.loader import DataLoader
import pandas as pd
from performance import performances_val
from PIL import Image
from torch.nn import Linear
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.data import Data
from sklearn import metrics
import numpy as np
from tqdm import tqdm
from torch_geometric.nn import ChebConv, GraphSAGE,GraphUNet ,TransformerConv
import cv2



# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
seed = 1378
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
landmark_colors = np.load("landmark_colors.npy")
from sklearn.preprocessing import OneHotEncoder
one_hot_list=[]
enc = OneHotEncoder()
enc.fit(np.expand_dims(landmark_colors,axis=1))  
for x in landmark_colors:
    one_hot_list.append(enc.transform([[x]]).toarray()[0])
one_hot_arr  = np.array(one_hot_list)
one_hot_arr = torch.FloatTensor(one_hot_arr)


In [2]:
class landmark_dataset(Dataset):
    def __init__(self, info_list, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform, pre_filter)
        
        self.graphs = pd.read_csv(info_list, delimiter=",", header=None)


    def len(self):
        return len(self.graphs)

    def get(self, idx):
        
        graph_path = self.graphs.iloc[idx]
        graph_path = graph_path[0]
        face_path = graph_path[:-3]
        
        face = cv2.imread(face_path)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        
        face = np.array(face,dtype =np.float32)
        face_features = DenseNet_feature_extractor_onnx(face)
        face_feature = torch.tensor(np.array(face_features), dtype=torch.float)
        
        data = torch.load(graph_path)
        #data.x =data.x.t()
        return Data(x=data.x, edge_index=data.edge_index,y =data.y,face_feature=face_feature) 
class landmark_dataset_train(Dataset):
    def __init__(self, info_list, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform, pre_filter)
        
        self.graphs = pd.read_csv(info_list, delimiter=",", header=None)


    def len(self):
        return len(self.graphs)

    def get(self, idx):
        
        graph_path = self.graphs.iloc[idx]
        graph_path = graph_path[0]
        
        
        face_path = graph_path[:-3]
        
        face = cv2.imread(face_path)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        
        face = np.array(face,dtype =np.float32)
        
        #face = np.expand_dims(face,axis=0)
        #print(face.shape)
        face = torch.tensor(face)
        
        #face = face.permute(2, 0, 1)
        
        face = torch.unsqueeze(face,dim =-1)
        
        
        data = torch.load(graph_path)
        
        domain_name = graph_path.split("/")[-4]
        
            
        if domain_name == "MSU_MFSD" :
            
            domain_label = torch.FloatTensor([[0,0,1]])
            
        elif domain_name == "Replay_attack_dataset" :
            
            domain_label = torch.FloatTensor([[0,1,0]])
            
        elif domain_name == "oulu" :
            
            domain_label = torch.FloatTensor([[0,0,1]])
        else :
            print(graph_path,domain_name )
            raise "error"
            
        
        data = Data(x=data.x, edge_index=data.edge_index,y =data.y,data = domain_label,y_node = one_hot_arr,face=face)

        #data.x =data.x.t()
        return data 


In [3]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
providers = ['CUDAExecutionProvider']

def preprocess_input_numpy(input_images):
    # اطمینان از اینکه تصاویر به float32 تبدیل شده‌اند
    input_images = input_images.astype(np.float32)
    
    # تبدیل محدوده پیکسل‌ها از [0, 255] به [0, 1]
    input_images /= 255.0

    # تبدیل محدوده پیکسل‌ها از [0, 1] به [-1, 1]
    input_images = input_images * 2.0 - 1.0

    return input_images


def DenseNet_feature_extractor_onnx(input_images):
    # پیش‌پردازش ورودی
    input_images = preprocess_input_numpy(input_images)

    # بررسی اینکه ورودی چهار بعدی است
    assert input_images.ndim == 4, "Input must have 4 dimensions (batch_size, height, width, channels)"

    # تهیه دیکشنری ورودی برای ONNX
    inputs = {ort_session.get_inputs()[0].name: input_images}
    
    # انجام پیش‌بینی
    onnx_outputs = ort_session.run(None, inputs)
    
    # استخراج ویژگی‌ها از خروجی‌های ONNX
    feature_extracted_0 = onnx_outputs[0]
    feature_extracted_1 = onnx_outputs[1]

    # محاسبه Global Average Pooling و Flatten برای هر تصویر در مینی‌بتچ
    global_pooling_0 = np.mean(feature_extracted_0, axis=(1, 2))
    global_pooling_1 = np.mean(feature_extracted_1, axis=(1, 2))

    flatten_0 = global_pooling_0.reshape(global_pooling_0.shape[0], -1)
    flatten_1 = global_pooling_1.reshape(global_pooling_1.shape[0], -1)

    # ادغام ویژگی‌ها
    total_feature = np.concatenate([flatten_0, flatten_1], axis=1)
    return total_feature


# Example usage
# input_image = tf.random.uniform((256, 256, 3))  # Replace with actual image data
# features = DenseNet_feature_extractor_onnx(input_image)
# print(features.shape)


# بارگذاری مدل ONNX
onnx_model_path = "/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.onnx"
ort_session = ort.InferenceSession(onnx_model_path , providers=providers)

# تابعی برای پیش‌پردازش ورودی مشابه DenseNet اما با استفاده از NumPy

# تابعی برای استفاده از مدل ONNX

# مثال استفاده
input_images = np.random.uniform(0, 255, (10, 256, 256, 3)).astype(np.uint8)  # جایگزین با داده واقعی
features = DenseNet_feature_extractor_onnx(input_images)
print(features.shape)  # باید به شکل (10, feature_length) 

/home/hassan-hossein/.local/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


(10, 640)


In [4]:
providers = [("CUDAExecutionProvider", {"device_id": torch.cuda.current_device(),
                                        "user_compute_stream": str(torch.cuda.current_stream().cuda_stream)})]
sess_options = ort.SessionOptions()
sess = ort.InferenceSession(onnx_model_path, sess_options=sess_options, providers=providers)


In [5]:

# تنظیمات برای استفاده از GPU


# # ایجاد جلسه استنتاج با استفاده از GPU
# ort_session = ort.InferenceSession("path_to_your_model.onnx", providers=providers)

# بررسی اینکه آیا GPU استفاده می‌شود یا خیر
print("Available providers:", ort_session.get_providers())
print("Current provider:", ort_session.get_provider_options())


Available providers: ['CPUExecutionProvider']
Current provider: {'CPUExecutionProvider': {}}


In [6]:
print(sess.get_providers())

['CPUExecutionProvider']


In [7]:
import onnxruntime as ort

# تنظیمات برای استفاده از GPU
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']

# ایجاد جلسه استنتاج با استفاده از GPU
ort_session = ort.InferenceSession(onnx_model_path , providers=providers)

print("Available providers:", ort_session.get_providers())


Available providers: ['CPUExecutionProvider']


In [ ]:
while True :
    features = DenseNet_feature_extractor_onnx(input_images)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# فرض کنید که داده‌های ورودی شما یک توالی از ۶۸۰ ویژگی دارد.
#sequence_length = 128
#feature_dim = 4  # هر ویژگی تک‌بعدی است

# نمونه داده ورودی (تعداد نمونه‌ها × طول توالی × بعد ویژگی)
#inputs = torch.rand(32, sequence_length, feature_dim)  # 32 نمونه با 680 ویژگی تک‌بعدی

# تعریف یک لایه ترنسفورمر انکودر
class SimpleTransformer(nn.Module):
    def __init__(self, embed_dim, num_layers, nhead):
        super(SimpleTransformer, self).__init__()
        self.linear = nn.Linear(1, embed_dim)  # افزایش بعد ویژگی‌ها
        self.encoder_layer = TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(768 * embed_dim, 1)  # فرضا 10 کلاس خروجی داریم

    def forward(self, x):
        x = self.linear(x)  # تغییر بعد ویژگی‌ها
        x = self.transformer_encoder(x)
        x = x.view(x.size(0), -1)  # فشرده‌سازی به یک بردار برای طبقه‌بندی
        x = self.fc(x)
        return x

# # ایجاد مدل و ارسال ورودی
# # model = SimpleTransformer(embed_dim=embed_dim, num_layers=2, nhead=4)
# # output = model(inputs)

# #print(output.shape)
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# فرض کنید که داده‌های ورودی شما یک توالی از ۶۸۰ ویژگی دارد.
sequence_length = 128
feature_dim = 1  # هر ویژگی تک‌بعدی است

# نمونه داده ورودی (تعداد نمونه‌ها × طول توالی × بعد ویژگی)
#inputs = torch.rand(32, sequence_length, feature_dim)  # 32 نمونه با 680 ویژگی تک‌بعدی

# تعریف یک لایه ترنسفورمر انکودر
class SimpleTransformer_2(nn.Module):
    def __init__(self, embed_dim, num_layers, nhead):
        super(SimpleTransformer_2, self).__init__()
        self.linear = nn.Linear(feature_dim, embed_dim)  # افزایش بعد ویژگی‌ها
        self.encoder_layer = TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(sequence_length * embed_dim, 7)  # فرضا 10 کلاس خروجی داریم

    def forward(self, x):
        x = self.linear(x)  # تغییر بعد ویژگی‌ها
        x = self.transformer_encoder(x)
        x = x.view(x.size(0), -1)  # فشرده‌سازی به یک بردار برای طبقه‌بندی
        x = self.fc(x)
        return x

# # ایجاد مدل و ارسال ورودی
# # model = SimpleTransformer(embed_dim=embed_dim, num_layers=2, nhead=4)
# # output = model(inputs)

# #print(output.shape)


In [ ]:
train_dataset = landmark_dataset_train("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/graph_train_O_M_I.txt")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=0)

test_dataset = landmark_dataset("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/graph_test_C.txt")
test_loader = DataLoader(test_dataset, batch_size=64,num_workers=16)


In [ ]:
for i in train_loader :
    print(i)
    break

In [ ]:
class RandomNodeDropout(nn.Module):
    def __init__(self, drop_prob=0.3):
        super(RandomNodeDropout, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x, enabled=True):
        if not self.training or not enabled:
            return x
        
        num_rows = x.size(0)
        mask = torch.rand(num_rows, device=x.device) > self.drop_prob
        mask = mask.float().unsqueeze(1).expand_as(x)
        x = x * mask
        
        return x

soft = torch.nn.Softmax(dim=1)

num_domain = 3
class GRL(nn.Module):

    def __init__(self, max_iter):
        super(GRL, self).__init__()
        self.iter_num = 0
        self.alpha = 10
        self.low = 0.0
        self.high = 1.0
        self.max_iter = max_iter

    def forward(self, input):
        self.iter_num += 1
        return input * 1.0

    def backward(self, gradOutput):
        coeff = np.float(2.0 * (self.high - self.low) / (1.0 + np.exp(-self.alpha * self.iter_num / self.max_iter))
                         - (self.high - self.low) + self.low)
        return -coeff*10* gradOutput

num_node  =train_dataset.num_node_features
class Discriminator(torch.nn.Module):
    def __init__(self,hidden_channels ,max_iter):
        super(Discriminator, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv2 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv3 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv4 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.grl_layer = GRL(max_iter)
        self.fc2 = nn.Linear(hidden_channels, num_domain)
        
    def forward(self, x, edge_index, batch,drop_en):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        
        x = self.grl_layer(x)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.drop_1(x,drop_en)
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.drop_2(x,drop_en)
        
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        
        x = global_mean_pool(x, batch)
        
        x = self.fc2(x)
         
        return x 
class Discriminator_G(nn.Module):
    def __init__(self, max_iter):
        super(Discriminator_G, self).__init__()
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, num_domain)
        self.ad_net = nn.Sequential(
            self.fc1,
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            self.fc2
        )
        self.grl_layer = GRL(max_iter)

    def forward(self, feature):
        adversarial_out = self.ad_net(self.grl_layer(feature))
        return adversarial_out


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels,max_iter=4000):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = ChebConv(num_node, hidden_channels,2)
        self.conv2 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv3 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv4 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv5 = ChebConv(hidden_channels*2, hidden_channels,2)
        self.conv6 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv7 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv8 = ChebConv(hidden_channels, hidden_channels,2)
        self.lin = SimpleTransformer(8,5,8)
        
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.drop_3 = RandomNodeDropout(drop_prob=0.3)
        self.drop_4 = RandomNodeDropout(drop_prob=0.3)
        self.drop_5 = RandomNodeDropout(drop_prob=0.3)
        self.drop_6 = RandomNodeDropout(drop_prob=0.3)
        
        
        self.b1 = BatchNorm(hidden_channels)
        self.b2 = BatchNorm(hidden_channels)
        self.b3 = BatchNorm(hidden_channels)
        self.b4 = BatchNorm(hidden_channels)
        self.b5 = BatchNorm(hidden_channels)
        self.b6 = BatchNorm(hidden_channels)
        self.b7 = BatchNorm(hidden_channels)
        
        self.dis = Discriminator(hidden_channels,max_iter)
        self.dis_g = Discriminator_G(max_iter)
        
        self.conv1_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.conv2_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.lin_node_class = SimpleTransformer_2(64,10,8)
        

        
        #self.tcn = Simple1DCNN()
        
    def forward(self, x, edge_index, batch,drop_en,index_arr,face_feature):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        

        x = self.conv1(x, edge_index)
        x  = self.b1(x)
        x = x.relu()
        
        x = self.drop_1(x,drop_en)
        
        

        x = self.conv2(x, edge_index)
        x  = self.b2(x)
        x = x.relu()
        
        x = self.drop_2(x,drop_en)
                
        dicriminator = x
        
        x_1 = global_mean_pool(x, batch)
        
        x = self.conv3(x, edge_index)
        x  = self.b3(x)
        x = x.relu()
        
        x = self.drop_3(x,drop_en)
        
        x_3 = global_mean_pool(x, batch)
        
        
        x = self.conv4(x, edge_index)
        x  = self.b4(x)
        x = x.relu()
        
        x = self.drop_4(x,drop_en)
        
        
        x = torch.cat((x,dicriminator),dim=1) #skip connection
        
        x = self.conv5(x, edge_index)
        x  = self.b5(x)
        x = x.relu()
        
        x = self.drop_5(x,drop_en)
        
        x_5 = global_mean_pool(x, batch)
        
        x = self.conv6(x, edge_index)
        x  = self.b6(x)
        x = x.relu()
        
        x = self.drop_6(x,drop_en)
        
        x = self.conv7(x, edge_index)
        x  = self.b7(x)
        x = x.relu()
        
        
        x = self.conv8(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        
        
        x = torch.cat((x,face_feature),dim=1)
                
        x = F.dropout(x, p=0.5, training=self.training)
        
        dis_g  = self.dis_g(x)
        
        x = torch.unsqueeze(x,2)
        #print(x.shape)
        
        x = self.lin(x)
        
        x= x.sigmoid()
        
        
        node_class_hidden = self.conv1_node_class(dicriminator,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        node_class_hidden = self.conv2_node_class(node_class_hidden,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        #print(index_arr.shape , node_class_hidden.shape)
        
        node_class_hidden = node_class_hidden[index_arr]
        
        #print(node_class_hidden.shape)
        node_class_hidden = torch.unsqueeze(node_class_hidden, dim=2)
        
        node_pre = self.lin_node_class(node_class_hidden)
        
            
        #print("hiiii :" , dicriminator.shape)
        dis_invariant = self.dis(dicriminator,edge_index,batch,drop_en)
        
        return x ,dis_invariant ,node_pre , dis_g

model = GCN(hidden_channels=128).to(device)
print(model)

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#criterion = torch.nn.CrossEntropyLoss()
criterion_domain_G = torch.nn.CrossEntropyLoss()
criterion= nn.BCELoss()
criterion_node_class=  torch.nn.CrossEntropyLoss()
criterion_2 = torch.nn.CrossEntropyLoss()
def train(epoch):
    model.train()
    losses_adv_loss=0
    losses_adv_g_loss=0
    losses_loss_cls=0
    total_loss = 0
    losses_node_cls_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}', unit='batch')
    for graph in progress_bar:  # Iterate in batches over the training dataset.
        
       
        face = np.moveaxis(graph.face.numpy(), -1, 0) 
        #print(face.shape)
        face_feature = torch.tensor(DenseNet_feature_extractor_onnx(face)).to(device)
        graph = graph.to(device)  # Move data to the device
        
        drop_en = bool(np.random.binomial(1, 0.5))
        
        if graph.x.shape[0] < 29952 :
            
            if graph.x.shape[0] < 64:
            
                index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
            else :
                
                index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
        else :
                
            index_arr = torch.randint(0, 29952, (64,)).to(device)
        
        
        
        out,domain_invariant,node_pre,dis_g = model(graph.x, graph.edge_index, graph.batch,drop_en,index_arr,face_feature)  # Perform a single forward pass.
        
        adv_loss = criterion_2(domain_invariant, graph.data)
        adv_g_loss = criterion_domain_G(dis_g, graph.data)
        
        node_cls_loss = criterion_node_class(node_pre.squeeze(),graph.y_node.float()[index_arr])
        #print(out.shape , graph.y.long().shape)
        loss_cls = criterion(out.squeeze(), graph.y.float())  # Compute the loss.
        
        loss_all = adv_loss + loss_cls + node_cls_loss + adv_g_loss
        loss_all.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        total_loss = loss_all + total_loss
        losses_adv_loss = adv_loss + losses_adv_loss
        losses_loss_cls = loss_cls + losses_loss_cls
        losses_node_cls_loss = node_cls_loss + losses_node_cls_loss
        losses_adv_g_loss  = adv_g_loss + losses_adv_g_loss

    print("/////////////////////////")
    print("adverserial loss : " ,losses_adv_loss/len(train_loader.dataset))  
    print("classification loss : " , losses_loss_cls/len(train_loader.dataset))
    print("classification node loss : " , losses_node_cls_loss/len(train_loader.dataset))
    print("losses_adv_g loss : " , losses_adv_g_loss/len(train_loader.dataset))
    print("total loss : " ,total_loss/len(train_loader.dataset) )
    print("////////////////////////////////////")
def test(loader,epoch):
    score_path = os.path.join("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores", "epoch_{}".format(epoch+1))
    check_folder(score_path)
    model.eval()
    with torch.no_grad():
        scores_list = []

        correct = 0
        s= 0
        progress_bar = tqdm(loader, desc=f'Epoch {epoch+1}', unit='batch')
        for graph in progress_bar:  # Iterate in batches over the training/test dataset.
            if graph.x.shape[0] < 29952 :

                if graph.x.shape[0] < 64:

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
                else :

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
            else :

                index_arr = torch.randint(0, 29952, (64,)).to(device)
                
            face_feature = DenseNet_feature_extractor_onnx(graph.face.numpy()).to(device)    
            graph = graph.to(device)  # Move data to the device
            logit,_ ,_= model(graph.x, graph.edge_index, graph.batch,False,index_arr,face_feature)
            live_label = graph.y.float()
            for i in range(len(logit)):
                        scores_list.append("{} {}\n".format(logit[i].item(), live_label[i].item()))
  # Check against ground-truth labels.
    map_score_val_filename = os.path.join(score_path, "score.txt")
    with open(map_score_val_filename, 'w') as file:
                file.writelines(scores_list)
    print("score: write test scores to {}".format(map_score_val_filename))
    test_ACC, fpr, FRR, HTER, auc_test, test_err, tpr = performances_val(map_score_val_filename)
    print("epoch:{:d}, test:  val_ACC={:.4f}, HTER={:.4f}, AUC={:.4f}, val_err={:.4f}, ACC={:.4f}, TPR={:.4f}".format(
        epoch + 1, test_ACC, HTER, auc_test, test_err, test_ACC, tpr))  
    return auc_test




In [ ]:
best_auc = 0
for epoch in range(1, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'add_garph_level_V_10_without_pipline_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue  

import tf2onnx
import tensorflow as tf

# Load your feature extractor model as defined in your code
# Assuming feature_extractor is already defined as in your provided code

# Specify the path where you want to save the ONNX model
onnx_model_path = "/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.onnx"

# Convert the Keras model to ONNX format
spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(feature_extractor, input_signature=spec, output_path=onnx_model_path)

print(f"Model converted to ONNX and saved at: {onnx_model_path}")


In [15]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
kernel_size = 20



model = DenseNet121(input_shape=(256,256,3),weights=None, include_top=False)

# Choose the desired low-level feature layers
low_level_layers = [
    model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
    model.get_layer('conv3_block12_concat')# Output of the last layer in the second dense block
]
# low_level_layers = [
#     model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
#     model.get_layer('conv3_block12_concat'), 
#     model.get_layer('conv4_block24_concat')# Output of the last layer in the second dense block
# ]

# Create a new model with the desired low-level feature layers as outputs
feature_extractor = tf.keras.Model(inputs=model.input, outputs=[layer.output for layer in low_level_layers])
feature_extractor.load_weights("/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.h5")
feature_extractor.trainable=False


#input_image = tf.random.uniform((10,40,40,3))
def DenseNet_feature_extractor(input_image) :
    #input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    #print("input shape", input_image.shape)
    input_image = tf.cast(input_image, tf.float32)
    input_image = tf.keras.applications.densenet.preprocess_input(input_image)
    #input_image = tf.expand_dims(input_image, axis=0)

    feature_extracted = feature_extractor.predict(input_image,verbose=False)
    #print(feature_extracted[0].shape,feature_extracted[1].shape)
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[0])
    flatten = tf.keras.layers.Flatten()(global_pooling)
    total_feature =flatten
    for layer_cnt in range(1,2) : 
        global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[layer_cnt])
        flatten = tf.keras.layers.Flatten()(global_pooling)
        total_feature = tf.concat([flatten,total_feature],1)
    return total_feature

In [16]:
import tensorflow as tf
import tf2onnx

# مسیر ذخیره مدل ONNX
onnx_model_path = "/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.onnx"

# تبدیل مدل Keras به فرمت ONNX
spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)

# تبدیل مدل به ONNX
model_proto, _ = tf2onnx.convert.from_keras(feature_extractor, input_signature=spec, output_path=onnx_model_path)

print(f"Model converted to ONNX and saved at: {onnx_model_path}")


Model converted to ONNX and saved at: /home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.onnx


In [8]:
باشد


(10, 640)


In [6]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Is CUDA available: True
CUDA devices: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3090


In [8]:
import torch

print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

CUDA version: 11.6


RuntimeError: cuDNN version incompatibility: PyTorch was compiled  against (8, 3, 2) but found runtime version (8, 1, 0). PyTorch already comes bundled with cuDNN. One option to resolving this error is to ensure PyTorch can find the bundled cuDNN.Looks like your LD_LIBRARY_PATH contains incompatible version of cudnnPlease either remove it from the path or install cudnn (8, 3, 2)